In [1]:
import json
import time
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pandas import *

In [2]:
#가게 이름, 가게 유형, 주소, 대표 메뉴 1개, 메뉴 설명, 리뷰 5개
def search_food(key):
    try:
        food_list = []
        
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome()  # 드라이버 경로
        # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
        driver.get(url)
        key_word = key  # 검색어

        # css 찾을때 까지 10초대기
        def time_wait(num, code):
            try:
                wait = WebDriverWait(driver, num).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, code)))
            except:
                print(code, '태그를 찾지 못하였습니다.')
                driver.quit()
            return wait

        # frame 변경 메소드
        def switch_frame(frame):
            driver.switch_to.default_content()  # frame 초기화"
            driver.switch_to.frame(frame)  # frame 변경

        # 페이지 다운
        def page_down(num):
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.click()
            for i in range(num):
                body.send_keys(Keys.PAGE_DOWN)

        # css를 찾을때 까지 10초 대기
        time_wait(10, 'div.input_box > input.input_search')

        # (1) 검색창 찾기
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(key_word)  # 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

        sleep(5)
        
        url_str = driver.current_url
        if url_str.find("isCorrectAnswer") == -1:
            print(url_str)
            print("결과 여러 개 존재(지점 또는 근접 지역)")
            
            search = driver.find_elements(By.ID, "searchIframe")
            # frame 변경
            switch_frame('searchIframe')
            sleep(2)
        
            
            tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')
            if tmp_shop == []:
                tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.place_bluelink')
            tmp_shop[0].click()
            
            sleep(5)
        
        else:
            print(url_str)
            print("결과 한 개")
        
        # (2) frame 변경
        switch_frame('entryIframe')
        #page_down(40)
        sleep(2)

        # 시작시간
        start = time.time()
        print('\n[크롤링 시작...]')
        
        shop_name = []
        shop_type = []
        shop_address = []
        menu_name = []
        menu_story = []
        review_1 = []
        review_2 = []
        review_3 = []
        review_4 = []
        review_5 = []
        
        place_star = driver.find_elements(By.CSS_SELECTOR, '.place_blind')
        
        
            
        
        #가게 이름
        names = driver.find_elements(By.CSS_SELECTOR, '.GHAhO')  # (3) 장소명
        shop_name.append(names[0].text)
        print(names[0].text)
        
        #가게 타입
        types = driver.find_elements(By.CSS_SELECTOR, '.lnJFt')  # (4) 장소 유형
        shop_type.append(types[0].text)
        print(types[0].text)
        
        #가게 주소
        address = driver.find_elements(By.CSS_SELECTOR, '.LDgIH') # (5) 주소
        shop_address.append(address[0].text)
        print(address[0].text)
        
        #버튼 정의
        home_list = driver.find_elements(By.CSS_SELECTOR, ".veBoZ") # (6) 리뷰
        
        #잠시 대기
        sleep(3)
        
        
        #메뉴 버튼 클릭
        for k in range(len(home_list)):
            if home_list[k].text == "리뷰":
                #print(home_list[k].text)
                home_list[k].click()
                sleep(3)
                break
            else:
                continue
        
        
        #리뷰 찾기
        revew_love_btn = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div[2]/div[1]/h2/span')
        
        if len(revew_love_btn) != 0:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            return tmp
        
        review_btn = driver.find_elements(By.CSS_SELECTOR, ".t3JSf") # (7) 리뷰
        
        #리뷰 1
        real_review = review_btn[0].text.split('"')
        review_1.append(real_review[1])
        print(real_review[1])
        #리뷰 2
        real_review = review_btn[1].text.split('"')
        review_2.append(real_review[1])
        print(real_review[1])
        #리뷰 3
        real_review = review_btn[2].text.split('"')
        review_3.append(real_review[1])
        print(real_review[1])
        #리뷰 4
        real_review = review_btn[3].text.split('"')
        review_4.append(real_review[1])
        print(real_review[1])
        #리뷰 5
        real_review = review_btn[4].text.split('"')
        review_5.append(real_review[1])
        print(real_review[1])
        
        tmp = DataFrame({'가게명': shop_name, '가게 유형' :shop_type, '가게 주소': shop_address, "리뷰1": review_1, "리뷰2": review_2, "리뷰3": review_3, "리뷰4": review_4,"리뷰5": review_5})

        print(f'{shop_name} ...완료')

        sleep(1)

        print('[데이터 수집 완료]\n소요 시간 :', time.time() - start, "\n")
        driver.quit()  # 작업이 끝나면 창을 닫는다.
        
        return tmp
    except:
         tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
         return tmp

In [3]:

regions = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", 
           "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]

#len(regions)
for i in range(len(regions)):
    tmp = DataFrame(columns = ['가게명', '가게 유형', '가게 주소', '리뷰1', '리뷰2', '리뷰3', '리뷰4', '리뷰5'])
    filename = "./out_play/out_" + str(i) + "_play.csv"
    df = read_csv(filename, header = None)
    df.columns = ['가게명']
    
    for j in range( len(df)):
        tp = df.loc[j]['가게명']
        key = "서울 " + regions[i] + " " + tp
        print(str(j) + "번째 가게")
        
        data = search_food(key)
        
        result2 = concat([tmp, data], ignore_index = True)
        tmp = result2

    #파일 저장
    filename = "./out_play_review/out_" + str(i) + "_play.csv"
    tmp.to_csv(filename, header = None, index = False)
    print(filename + "완료")


0번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EC%84%9C%EC%9A%B8%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%84%9C%EC%9A%B8%EC%97%AD%EC%82%AC%EB%B0%95%EB%AC%BC%EA%B4%80?c=15.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)

[크롤링 시작...]
서울역사박물관
박물관
서울 종로구 새문안로 55
유익해요
전시 구성이 알차요
설명이 잘 되어있어요
주차하기 편해요
전시 테마가 재밌어요
['서울역사박물관'] ...완료
[데이터 수집 완료]
소요 시간 : 7.264041423797607 

1번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EC%84%9C%EC%9A%B8%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EA%B2%BD%ED%9D%AC%EA%B6%81?c=15.00,0,0,0,dh
결과 여러 개 존재(지점 또는 근접 지역)

[크롤링 시작...]
경희궁
궁궐
서울 종로구 새문안로 45 경희궁.시립미술관
붐비지 않아요
사진이 잘 나와요
산책로가 잘 되어있어요
관리가 잘 되어있어요
아이와 가기 좋아요
['경희궁'] ...완료
[데이터 수집 완료]
소요 시간 : 7.2578301429748535 

2번째 가게


...

https://map.naver.com/p/search/%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%9E%87%EB%8B%A4/place/1642396952?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
잇다
카페,디저트
서울 강동구 천호대로 1156-5 1층, B1층
커피가 맛있어요
인테리어가 멋져요
친절해요
음료가 맛있어요
디저트가 맛있어요
['잇다'] ...완료
[데이터 수집 완료]
소요 시간 : 7.222506523132324 

109번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%BB%A4%ED%94%BC%EB%AA%BD%ED%83%80%EC%A3%BC/place/31905764?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
커피몽타주
카페
서울 강동구 올림픽로48길 23-12 1층
커피가 맛있어요
친절해요
매장이 청결해요
음료가 맛있어요
인테리어가 멋져요
['커피몽타주'] ...완료
[데이터 수집 완료]
소요 시간 : 7.218153715133667 

110번째 가게


The chromedriver version (128.0.6613.137) detected in PATH at c:\Users\kk21\Desktop\TRIPER\chromedriver.exe might not be compatible with the detected chrome version (129.0.6668.101); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


https://map.naver.com/p/search/%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%8F%99%EA%B5%AC%20%ED%95%B4%ED%94%BC%EB%B2%A0%EC%96%B4%EB%8D%B0%EC%9D%B4%20%ED%98%84%EB%8C%80%EB%B0%B1%ED%99%94%EC%A0%90%20%EC%B2%9C%ED%98%B8%EC%A0%90/place/1347769273?c=15.00,0,0,0,dh&isCorrectAnswer=true
결과 한 개

[크롤링 시작...]
해피베어데이 현대백화점 천호점
카페,디저트
서울 강동구 천호대로 1005 지하 2층 해피베어데이
디저트가 맛있어요
특별한 메뉴가 있어요
친절해요
선물하기 좋아요
사진이 잘 나와요
['해피베어데이 현대백화점 천호점'] ...완료
[데이터 수집 완료]
소요 시간 : 7.231740474700928 

./out_play_review/out_24_play.csv완료


In [ ]:
#지금 번호 확인
print(i)
print(j)

In [ ]:
# 합쳐야 할 때 맨 처음 한 번만 실행
# ex = tmp.copy()

In [ ]:
ex2 = tmp.copy()
ex2

In [ ]:
import pandas as pd
tmp_df = pd.concat([ex,ex2], axis=0, ignore_index=True)
tmp_df[-3:]

In [ ]:
ex = tmp_df.copy()
ex[-3:]